In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

# Check GPU availability

In [3]:
import torch

# Check if a GPU is available
gpu_available = torch.cuda.is_available()

if gpu_available:
    # Get the name of the GPU
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU is available. GPU Name: {gpu_name}")
else:
    print("GPU is not available. Running on CPU.")

GPU is available. GPU Name: Tesla P100-PCIE-16GB


# Find class counts

In [4]:
medical_data = pd.read_csv("/kaggle/input/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/patient_diagnosis.csv", names = ["patient", "diagnosis"])
print(medical_data.head())
print()
value_counts = medical_data["diagnosis"].value_counts()
print(value_counts)


   patient diagnosis
0      101      URTI
1      102   Healthy
2      103    Asthma
3      104      COPD
4      105      URTI

diagnosis
COPD              64
Healthy           26
URTI              14
Bronchiectasis     7
Bronchiolitis      6
Pneumonia          6
LRTI               2
Asthma             1
Name: count, dtype: int64


# How many cycles/samples are there in total?

In [5]:
import glob
directory_path = '/kaggle/input/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files'
text_files = glob.iglob(directory_path + '/*.txt', recursive=True)
text_files_list = list(text_files)

dfs = []
for file in text_files_list:
    content = pd.read_csv(file, names = ["start_time", "end_time", "crackles", "wheezes"], delimiter = "\t")
    patient_number = file.split("_")[7][6:]
    content['patient'] = int(patient_number)
    dfs.append(content)
samples = pd.concat(dfs)

print("There are", len(samples), "cycles")
print("There are", len(text_files_list), "samples")

There are 6898 cycles
There are 920 samples


# How many samples are there in each class?

In [6]:
all_data = pd.merge(medical_data, samples, on = "patient", how = "outer")
all_data["diagnosis"].value_counts()

diagnosis
COPD              5746
Healthy            322
Pneumonia          285
URTI               243
Bronchiolitis      160
Bronchiectasis     104
LRTI                32
Asthma               6
Name: count, dtype: int64

# Compute stats (mean, std dev) on the length of resp cycles (over all samples)

In [7]:
resp_cycles_len = samples["end_time"] - samples["start_time"]
print("Mean is", resp_cycles_len.mean())
print("Standard deviation is", resp_cycles_len.std())

Mean is 2.700509085242099
Standard deviation is 1.1725341343398874


# Clear the output folder

In [36]:
import os
import shutil

# clear output folder
output_folder = '/kaggle/working/'
for file_name in os.listdir(output_folder):
    file_path = os.path.join(output_folder, file_name)
    # Remove the file
    if os.path.isfile(file_path):
        os.remove(file_path)

print(f"Contents of {output_folder} cleared.")

# how many files in output folder?
output_folder = '/kaggle/working/'
files_in_output_folder = os.listdir(output_folder)
num_files = len(files_in_output_folder)
print("num files in output folder:", num_files)

Contents of /kaggle/working/ cleared.
num files in output folder: 1


# Generate spectrograms

**Define function to generate spectograms for one audio file**

In [31]:
import librosa as librosa
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
%matplotlib inline
import librosa.display
from IPython.display import Audio
import time
import gc


# converts file, where file is a string (name of the file), into a string with the given format, where format is a string
# assumes the format already in the file is 3 characters long
# if cycle > 0, it adds "cycle _" to the end of the file name before the ending
# if cycle = 0, it leaves the rest of the file as is
def convert_type(file, format, cycle):
    if cycle > 0:
        file = file[0:len(file)-4] + "_cycle" + str(cycle) + "." + format
    else:
        file = file[0:len(file)-3] + format
    return file


# divide audio_file into its respiratory cycles
# takes the name of the audio file (.wav) as a string as the argument
def divide_into_cycles(audio_file_name):
    # take just the timestamps for the respiratory cycles- ignore crackles and wheezes
    audio_file_name_as_txt = directory_path + "/" + convert_type(audio_file_name, "txt", 0)
    timestamps = np.loadtxt(audio_file_name_as_txt, delimiter='\t')
    timestamps = timestamps[:, :2]
    
    return timestamps


def generate_spectogram(audio_file):
    # load the audio file
    y, sr = librosa.load(audio_file)
    
    audio_file_name = os.path.basename(audio_file)
    # find where the timestamps of audio_file are
    timestamps = divide_into_cycles(audio_file_name) # list
    
    # make a mel db spectrogram for each respiratory cycle using the timestamps
    cycle_number = 1
    for cycle in timestamps:
        # take the portion of the audio that contains this respiratory cycle

        # start and end time in seconds
        start_time = cycle[0]
        end_time = cycle[1]
        
        # convert the start and end times to sample indices
        start_sample = int(start_time * sr)
        end_sample = int(end_time * sr)
        
        # extract the audio segment
        audio_segment = y[start_sample:end_sample]
        
        spectrogram_name = convert_type(audio_file_name, "jpg", cycle_number)
        
        # make the mel db spectrogram based on that audio segment
        D = librosa.amplitude_to_db(np.abs(librosa.stft(audio_segment, n_fft=2048, hop_length=512)), ref=np.max)
        # display the spectrogram
        plt.figure(figsize=(5, 2))
        librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log', cmap='viridis')
        plt.colorbar(format='%+2.0f dB')
        plt.title(spectrogram_name)

        # save the spectrogram to the output folder
        output_path = os.path.join(output_folder, spectrogram_name)
        plt.savefig(output_path)
        plt.close()
        
        cycle_number += 1

**Generate spectograms in batches**

In [37]:
audio_files = glob.iglob(directory_path + '/*.wav', recursive=True)

In [47]:
batch_size = 200
begin_time = time.time()
file_number = 0

try:
    for audio_file in audio_files:
        generate_spectogram(audio_file)
        file_number += 1

        # Print progress
        if file_number % 10 == 0:
            print(f"Processed {file_number} files so far in {time.time() - begin_time} seconds")
            gc.collect()

        if file_number >= batch_size:
            break
except StopIteration:
    print("No more audio files")

print(f"Processed a batch of {file_number} files in {time.time() - begin_time} seconds")

Processed 10 files so far in 21.861838340759277 seconds
Processed a batch of 18 files in 44.982096433639526 seconds


**Copy Diagnosis file to output**

In [19]:
import shutil
shutil.copy("/kaggle/input/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/patient_diagnosis.csv",
           "/kaggle/working/")

0